In [2]:
import wikipediaapi

def extract_family_relationships(page, current_depth, max_depth, visited, relationships):
    if current_depth > max_depth or page.title in visited:
        return
    visited.add(page.title)

    # Check page sections for key family-related terms and extract linked languages
    # Simple heuristic: look for sections like "Classification", "Family", "Ancestor", "Descendants"
    for section in page.sections:
        title_lower = section.title.lower()
        if any(keyword in title_lower for keyword in ['family', 'classification', 'ancestor', 'descendant', 'dialect']):
            # Extract linked languages from section text
            text_links = page.links
            for link_name in text_links:
                if link_name.lower() != page.title.lower():
                    # Relationship type based on section title
                    if "ancestor" in title_lower:
                        relation = "ancestor of"
                        relationships.add((link_name, relation, page.title))
                    elif "descendant" in title_lower or "dialect" in title_lower:
                        relation = "descendant of"
                        relationships.add((page.title, relation, link_name))
                    else:
                        relation = "related to"
                        relationships.add((page.title, relation, link_name))

                    # Recursive traversal
                    linked_page = wiki_wiki.page(link_name)
                    if linked_page.exists():
                        extract_family_relationships(linked_page, current_depth+1, max_depth, visited, relationships)

def get_language_family_tree(lang_name, depth):
    global wiki_wiki
    wiki_wiki = wikipediaapi.Wikipedia(user_agent='LanguageFamilyTreeBot (example@example.com)', language='en')
    relationships = set()
    visited = set()

    page = wiki_wiki.page(lang_name)
    if not page.exists():
        print(f"Wikipedia page for '{lang_name}' not found.")
        return []

    # Extract relationships including from the main page summary and sections
    extract_family_relationships(page, 0, depth, visited, relationships)

    return relationships

def main():
    lang = input("Enter the language name (as in Wikipedia page title): ")
    depth = int(input("Enter the depth for traversal (e.g. 2): "))
    results = get_language_family_tree(lang, depth)
    if results:
        print("\nRelationships found:")
        for rel in results:
            print(rel)
    else:
        print("No relationships found.")

if __name__ == "__main__":
    main()


No relationships found.
